# Retrieving Data

In [1]:
# Install required packages in the folllowing few cells

In [2]:
%%capture
 # capture hides output
!pip install yfinance pandas numpy scikit-learn ta pandas_ta xgboost

In [3]:
%%capture
# hidden output
!pip install alpha_vantage

In [4]:
%%capture
# hidden output, !pip install is required to run just on a machine
!pip install requests pandas_datareader


#### IMPORTS

In [10]:
import numpy as np
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
from datetime import date
from datetime import datetime
from datetime import time
from datetime import timedelta
from pandas.tseries.offsets import BDay
import matplotlib.lines as mlines
from dotenv import load_dotenv
import os
import matplotlib.patches as patches
import seaborn as sns
import warnings
import requests
import json
import pandas_ta as ta
from ta import trend, volatility, momentum, volume
import yfinance as yf
import time

from numpy import array
from importlib import reload # to reload modules if we made changes to them without restarting kernel


# Source Data

In [6]:
# Define the function to get the sector for each stock
def get_sector(ticker):
    try:
        stock_info = yf.Ticker(ticker).info
        return stock_info.get('sector', 'N/A')  # Returns 'N/A' if sector information is not available
    except Exception as e:
        print(f"Could not fetch sector for {ticker}: {e}")
        return 'N/A'
    

# get ticker symbols
tickers = pd.read_csv('data/tickers.csv')
tickers = tickers['Symbol'].tolist()

sectors = []

# Loop through each ticker and fetch the sector
for ticker in tickers:
    print(f"Fetching sector for {ticker}")
    sector = get_sector(ticker)
    sectors.append(sector)
   # time.sleep(0.5)  # Delay to avoid hitting API rate limits

Fetching sector for MSFT
Fetching sector for AAPL
Fetching sector for NVDA
Fetching sector for AMZN
Fetching sector for GOOGL
Fetching sector for META
Fetching sector for GOOG
Fetching sector for BRK.B
Fetching sector for AVGO
Fetching sector for LLY
Fetching sector for JPM
Fetching sector for XOM
Fetching sector for TSLA
Fetching sector for UNH
Fetching sector for V
Fetching sector for MA
Fetching sector for PG
Fetching sector for JNJ
Fetching sector for HD
Fetching sector for MRK
Fetching sector for COST
Fetching sector for CVX
Fetching sector for ABBV
Fetching sector for CRM
Fetching sector for BAC
Fetching sector for WMT
Fetching sector for AMD
Fetching sector for NFLX
Fetching sector for PEP
Fetching sector for KO
Fetching sector for TMO
Fetching sector for ADBE
Fetching sector for WFC
Fetching sector for LIN
Fetching sector for DIS
Fetching sector for MCD
Fetching sector for CSCO
Fetching sector for ACN
Fetching sector for ORCL
Fetching sector for ABT
Fetching sector for QCOM
Fet

In [7]:
set(sectors)


{'Basic Materials',
 'Communication Services',
 'Consumer Cyclical',
 'Consumer Defensive',
 'Energy',
 'Financial Services',
 'Healthcare',
 'Industrials',
 'N/A',
 'Real Estate',
 'Technology',
 'Utilities'}

In [8]:
# get tickers
tickers = pd.read_csv('data/tickers.csv')

tickers['Sector'] = sectors

# Check the result
print(tickers.head())

# Save the result to a CSV file
tickers.to_csv('data/tickers.csv', index=False)

   #                Company Symbol Portfolio%                  Sector
0  1         Microsoft Corp   MSFT      7.05%              Technology
1  2             Apple Inc.   AAPL      5.74%              Technology
2  3            Nvidia Corp   NVDA      5.06%              Technology
3  4         Amazon.com Inc   AMZN      3.83%       Consumer Cyclical
4  5  Alphabet Inc. Class A  GOOGL      2.37%  Communication Services


In [15]:
tickers = pd.read_csv('data/tickers.csv')[0:300]
#get ticker symbols
ticker_symbols = tickers['Symbol'].tolist()

In [16]:
tickers

,#,Company,Symbol,Portfolio%,Sector
0,1,Microsoft Corp,MSFT,7.05%,Technology
1,2,Apple Inc.,AAPL,5.74%,Technology
2,3,Nvidia Corp,NVDA,5.06%,Technology
3,4,Amazon.com Inc,AMZN,3.83%,Consumer Cyclical
4,5,Alphabet Inc. Class A,GOOGL,2.37%,Communication Services
...,...,...,...,...,...
295,296,"Cardinal Health, Inc.",CAH,0.06%,Healthcare
296,297,Dollar Tree Inc.,DLTR,0.06%,Consumer Defensive
297,298,Dover Corporation,DOV,0.06%,Industrials
298,299,Fifth Third Bancorp,FITB,0.06%,Financial Services


In [11]:
ticker_symbols

['MSFT',
 'AAPL',
 'NVDA',
 'AMZN',
 'GOOGL',
 'META',
 'GOOG',
 'BRK.B',
 'AVGO',
 'LLY',
 'JPM',
 'XOM',
 'TSLA',
 'UNH',
 'V',
 'MA',
 'PG',
 'JNJ',
 'HD',
 'MRK',
 'COST',
 'CVX',
 'ABBV',
 'CRM',
 'BAC',
 'WMT',
 'AMD',
 'NFLX',
 'PEP',
 'KO',
 'TMO',
 'ADBE',
 'WFC',
 'LIN',
 'DIS',
 'MCD',
 'CSCO',
 'ACN',
 'ORCL',
 'ABT',
 'QCOM',
 'INTU',
 'GE',
 'CAT',
 'AMAT',
 'VZ',
 'DHR',
 'TXN',
 'COP',
 'IBM',
 'CMCSA',
 'NOW',
 'UNP',
 'PM',
 'AMGN',
 'PFE',
 'UBER',
 'GS',
 'NEE',
 'INTC',
 'AXP',
 'RTX',
 'SPGI',
 'LOW',
 'ISRG',
 'ETN',
 'HON',
 'MU',
 'ELV',
 'BKNG',
 'PGR',
 'LRCX',
 'T',
 'C',
 'MS',
 'NKE',
 'SYK',
 'SCHW',
 'TJX',
 'BSX',
 'UPS',
 'BLK',
 'MDT',
 'CI',
 'DE',
 'VRTX',
 'ADI',
 'SBUX',
 'ADP',
 'CB',
 'LMT',
 'MMC',
 'PLD',
 'KLAC',
 'BA',
 'MDLZ',
 'FI',
 'REGN',
 'PANW',
 'BMY',
 'BX',
 'CMG',
 'TMUS',
 'CVS',
 'SNPS',
 'GILD',
 'AMT',
 'SO',
 'EOG',
 'WM',
 'CDNS',
 'MO',
 'TGT',
 'CME',
 'ICE',
 'DUK',
 'CL',
 'MPC',
 'ZTS',
 'FCX',
 'SHW',
 'APH',
 'MCK',
 

In [19]:
# Split tickers into batches of 100 for we don't hit the API rate limit
ticker_batches = [ticker_symbols[i:i + 100] for i in range(0, len(ticker_symbols), 100)]

# Start and end dates
start_date = '2005-01-01'
end_date = '2024-01-01'

# Create an empty DataFrame to store all data
all_data = pd.DataFrame()

# Loop through each batch of tickers and fetch the historical data
for batch in ticker_batches:
    for ticker in batch:
        print(f"Downloading data for {ticker}")
        data = yf.download(ticker, start=start_date, end=end_date, interval="1wk")
        data.dropna(inplace=True)  # Clean data by dropping NAs
        data['Ticker'] = ticker  # Add a column to identify the stock
        #data['Sector'] = get_sector(ticker) # Add a column to identify the sector
        all_data = pd.concat([all_data, data], axis=0)
    
    # Save to CSV after each batch is processed
    #all_data.to_csv('data/historical_stock_data_partial.csv')

    time.sleep(10)  # sleep tim to avoid hitting API rate limits

# Save the final result to a CSV file
all_data.to_csv('data/historical_stock_data_final.csv')

print("All data downloaded and saved.")


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')
[*********************100%%**********************]  1 of 1 completed


...

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


All data downloaded and saved.


# FEATURE ENGINEERING


### TECHNICAL INDICATORS:

* MACD (Moving Average Convergence Divergence)
* RSI (Relative Strength Index)
* Bollinger Bands High (the low is commented out)
* ADX (Average Directional Index)
* ATR (Average True Range)
* CCI (Commodity Channel Index)
* MFI (Money Flow Index)
* Stochastic Oscillator
* Keltner Channel
* VWAP (Volume Weighted Average Price)
* EOM (Ease Of Movement)
* VPT (Volume Price Trend)

In [20]:
# 1. Load data
data_yf = pd.read_csv('data/historical_stock_data_final.csv')

In [21]:
data_yf

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker
0,2005-01-01,26.799999,27.100000,26.620001,26.670000,18.611700,392522300.0,MSFT
1,2005-01-08,26.600000,26.860001,26.040001,26.120001,18.227880,390071600.0,MSFT
2,2005-01-15,26.030001,26.350000,25.639999,25.650000,17.899891,262141600.0,MSFT
3,2005-01-22,25.760000,26.650000,25.639999,26.180000,18.269747,405236700.0,MSFT
4,2005-01-29,26.350000,26.520000,26.100000,26.320000,18.367451,332545200.0,MSFT
...,...,...,...,...,...,...,...,...
272515,2023-11-27,404.010010,405.029999,392.950012,401.869995,401.869995,4697800.0,CHTR
272516,2023-12-04,400.649994,407.769989,358.480011,367.549988,367.549988,7812600.0,CHTR
272517,2023-12-11,372.589996,395.019989,364.399994,379.929993,379.929993,6791000.0,CHTR
272518,2023-12-18,381.579987,391.279999,378.529999,382.089996,382.089996,3555900.0,CHTR


In [22]:
# 2. Clean data_yf: DROP NAs
data_yf.dropna(inplace=True)

# 3. Add technical indicators

data_yf['macd'] = trend.MACD(data_yf['Close']).macd()
data_yf['macd_signal'] = trend.MACD(data_yf['Close']).macd_signal()
data_yf['rsi'] = momentum.RSIIndicator(data_yf['Close']).rsi()
data_yf['bb_high'] = volatility.BollingerBands(data_yf['Close']).bollinger_hband()
#data_yf['bb_low'] = volatility.BollingerBands(data_yf['Close']).bollinger_lband()
data_yf['adx'] = trend.ADXIndicator(data_yf['High'], data_yf['Low'], data_yf['Close']).adx()
data_yf['atr'] = volatility.AverageTrueRange(data_yf['High'], data_yf['Low'], data_yf['Close']).average_true_range()
data_yf['cci'] = trend.CCIIndicator(data_yf['High'], data_yf['Low'], data_yf['Close']).cci()
data_yf['mfi'] = volume.MFIIndicator(data_yf['High'], data_yf['Low'], data_yf['Close'], data_yf['Volume']).money_flow_index()
data_yf['so'] = momentum.StochasticOscillator(data_yf['High'], data_yf['Low'], data_yf['Close']).stoch()
# Calculate additional indicators
data_yf['kc'] = volatility.KeltnerChannel(data_yf['High'], data_yf['Low'], data_yf['Close']).keltner_channel_mband()
data_yf['vwap'] = volume.VolumeWeightedAveragePrice(data_yf['High'], data_yf['Low'], data_yf['Close'], data_yf['Volume']).volume_weighted_average_price()
data_yf['eom'] = volume.EaseOfMovementIndicator(data_yf['High'], data_yf['Low'], data_yf['Close'], data_yf['Volume']).ease_of_movement()
data_yf['vpt'] = volume.VolumePriceTrendIndicator(data_yf['Close'], data_yf['Volume']).volume_price_trend()


# Momentum Indicators
data_yf.ta.rsi(append=True)
data_yf.ta.mfi(append=True)
data_yf.ta.roc(append=True)
data_yf.ta.willr(append=True)

# Volume Indicators
data_yf.ta.obv(append=True)


# Volatility Indicators
# pandas_ta doesn't seem to have NATR equivalent, but ATR is available

# Average True Range (ATR)
# ATR is a technical analysis indicator that measures market volatility by decomposing the entire range of an asset price for that period.
# It is commonly used to understand the volatility of a stock, commodity, forex, or crypto over a given period.
# The ATR is particularly useful in systems that need to place actual numerical values to the volatility or potential price change of an asset, often used to adjust trading systems and stop-loss orders.

# Calculate ATR and add it to the DataFrame
data_yf['atr'] = data_yf.ta.atr()

# 4. Create Buy/Sell signals based on the technical indicators

# Either use the indicator values directly or create signals based on the indicator values
# depending on the strategy, we can create signals for each indicator!!
# these signals are used to create a trading strategy and are very subjective; they might not even be useful
# we can get rid of them.

# SIGNALS for each indicator
# Calculate the Force Index (FI)
# data_yf['fi'] = (data_yf['Close'].diff()) * data_yf['Volume']

# # Create a Buy/Sell/Hold Signal for FI
# data_yf['fi_buy_signal'] = np.where(data_yf['fi'] > 0, 1, 0)  # Buy if FI is positive
# data_yf['fi_sell_signal'] = np.where(data_yf['fi'] < 0, 1, 0)  # Sell if FI is negative

# # MACD signal
# data_yf['macd_buy_signal'] = np.where(data_yf['macd'] > 0, 1, 0)
# data_yf['macd_sell_signal'] = np.where(data_yf['macd'] < 0, 1, 0)

# # RSI signal
# data_yf['rsi_buy_signal'] = np.where(data_yf['rsi'] < 30, 1, 0)
# data_yf['rsi_sell_signal'] = np.where(data_yf['rsi'] > 70, 1, 0)

# # Stochastic Oscillator signal
# data_yf['so_buy_signal'] = np.where(data_yf['so'] < 20, 1, 0)
# data_yf['so_sell_signal'] = np.where(data_yf['so'] > 80, 1, 0)

# # CCI signal
# data_yf['cci_buy_signal'] = np.where(data_yf['cci'] < -100, 1, 0)
# data_yf['cci_sell_signal'] = np.where(data_yf['cci'] > 100, 1, 0)

# # MFI signal
# data_yf['mfi_buy_signal'] = np.where(data_yf['mfi'] < 20, 1, 0)
# data_yf['mfi_sell_signal'] = np.where(data_yf['mfi'] > 80, 1, 0)

# # WILLR signal
# data_yf['WILLR_buy_signal'] = np.where(data_yf['WILLR_14'] < -80, 1, 0)
# data_yf['WILLR_sell_signal'] = np.where(data_yf['WILLR_14'] > -20, 1, 0)


# Other indicators such as ATR, EOM, VWAP, etc., may not have clear buy/sell interpretations.
# Depending on strategy: might want to create signals for these as well.

# If suing signals, drop the original indicator columns
# if not using signals, keep the original indicator columns

# Drop the original technical indicator columns, keeping only the signal columns
# indicator_cols_to_drop = [
#     'macd', 'macd_signal', 'rsi', 'bb_high', 'adx', 'atr', 'cci', 'mfi', 'so', 'kc', 'fi', 'vwap', 'eom', 'vpt',
#     'WILLR_14', 'OBV' # pandas_ta indicators
# ]
# data_yf.drop(columns=indicator_cols_to_drop, inplace=True)

In [23]:
data_yf.columns


Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Ticker',
       'macd', 'macd_signal', 'rsi', 'bb_high', 'adx', 'atr', 'cci', 'mfi',
       'so', 'kc', 'vwap', 'eom', 'vpt', 'RSI_14', 'MFI_14', 'ROC_10',
       'WILLR_14', 'OBV'],
      dtype='object')

In [24]:
data_yf.shape

(272520, 26)

#### MOVING AVERAGES

In [25]:
# Create lagged features for each indicator for the previous 5 days
indicator_cols = [col for col in data_yf.columns if col not in ['win', 'gain_loss_pct', 'Open', 'High', 'Low', 'Close', 'Volume','Adj Close']]

for col in indicator_cols:
    for i in range(1, 6):
        data_yf[f'{col}_lag_{i}'] = data_yf[col].shift(i)

# Remove rows with NaN values after shifting
data_yf.dropna(inplace=True)


In [26]:
# 4. Calculate gain/loss percentage and assign -1, 0, 1
data_yf['gain_loss_pct'] = (data_yf['Close'] - data_yf['Open']) / data_yf['Open']
data_yf['win'] = np.where(data_yf['gain_loss_pct'] > 0.005, 1, 
                          np.where(data_yf['gain_loss_pct'] < -0.0025, -1, 0)) # 1 if 0.5% gain, -1 if 0.25% loss, 0 otherwise


# Drop OHLCV columns
#data_yf.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close'], inplace=True)


In [27]:
data_yf.shape, data_yf.columns, data_yf.describe()

((272482, 128),
 Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Ticker',
        'macd', 'macd_signal',
        ...
        'WILLR_14_lag_3', 'WILLR_14_lag_4', 'WILLR_14_lag_5', 'OBV_lag_1',
        'OBV_lag_2', 'OBV_lag_3', 'OBV_lag_4', 'OBV_lag_5', 'gain_loss_pct',
        'win'],
       dtype='object', length=128),
                 Open           High            Low          Close  \
 count  272482.000000  272482.000000  272482.000000  272482.000000   
 mean       94.572516      97.236772      92.013563      94.801635   
 std       149.400937     153.746156     145.428462     149.873351   
 min         0.353125       0.388021       0.337188       0.352708   
 25%        31.650000      32.596589      30.715703      31.700001   
 50%        55.930000      57.410000      54.410000      56.020000   
 75%       101.550003     104.339996      98.959999     101.858322   
 max      3527.469971    3580.620117    3525.290039    3547.219971   
 
            Adj Close   

In [28]:
# create csv with features data
data_yf.to_csv('data/all_stock_data.csv', index=False)

In [29]:
data_yf = pd.read_csv('data/all_stock_data.csv')

In [30]:
from sklearn.preprocessing import StandardScaler

float_columns = data_yf.select_dtypes(include=['float64']).columns

# Initialize the StandardScaler
scaler = StandardScaler()

# Standardize only the float columns
data_yf[float_columns] = scaler.fit_transform(data_yf[float_columns])

In [31]:
data_yf.isna().sum()

Date             0
Open             0
High             0
Low              0
Close            0
                ..
OBV_lag_3        0
OBV_lag_4        0
OBV_lag_5        0
gain_loss_pct    0
win              0
Length: 128, dtype: int64

In [32]:
# create csv with features data
data_yf.to_csv('data/all_stock_data.csv', index=False)